In [4]:
import os
import psycopg
from dotenv import load_dotenv

In [5]:
# Load environment variables from .env file
load_dotenv('../.env')
neon_conn_string = os.getenv("NEON_DB_URL")

In [4]:
try:
    with psycopg.connect(neon_conn_string) as neon_conn:
        print("Connection established")
        # Open a cursor to perform database operations
        with neon_conn.cursor() as cur:
            # Drop the table if it already exists
            cur.execute("DROP TABLE IF EXISTS books;")
            print("Finished dropping table (if it existed).")
            # Create a new table
            cur.execute("""
                CREATE TABLE books (
                    id SERIAL PRIMARY KEY,
                    title VARCHAR(255) NOT NULL,
                    author VARCHAR(255),
                    publication_year INT,
                    in_stock BOOLEAN DEFAULT TRUE
                );
            """)
            print("Finished creating table.")
            # Insert a single book record
            cur.execute(
                "INSERT INTO books (title, author, publication_year, in_stock) VALUES (%s, %s, %s, %s);",
                ("The Catcher in the Rye", "J.D. Salinger", 1951, True),
            )
            print("Inserted a single book.")
            # Data to be inserted
            books_to_insert = [
                ("The Hobbit", "J.R.R. Tolkien", 1937, True),
                ("1984", "George Orwell", 1949, True),
                ("Dune", "Frank Herbert", 1965, False),
            ]
            # Insert multiple books at once
            cur.executemany(
                "INSERT INTO books (title, author, publication_year, in_stock) VALUES (%s, %s, %s, %s);",
                books_to_insert,
            )
            print("Inserted 3 rows of data.")
            # The transaction is committed automatically when the 'with' block exits in psycopg (v3)
except Exception as e:
    print("Connection failed.")
    print(e)

Connection established
Finished dropping table (if it existed).
Finished creating table.
Inserted a single book.
Inserted 3 rows of data.


In [5]:
try:
    with psycopg.connect(neon_conn_string) as neon_conn:
        print("Connection established")
        # Open a cursor to perform database operations
        with neon_conn.cursor() as cur:
            cur.execute("SELECT * FROM books")
            print(cur.fetchall())
except Exception as e:
    print("Connection failed.")
    print(e)

Connection established
[(1, 'The Catcher in the Rye', 'J.D. Salinger', 1951, True), (2, 'The Hobbit', 'J.R.R. Tolkien', 1937, True), (3, '1984', 'George Orwell', 1949, True), (4, 'Dune', 'Frank Herbert', 1965, False)]


In [6]:
import pandas as pd

## Rooms table

First we need the rooms table as its primary key is referenced by the room_availability table.

In [157]:
df_rooms = pd.read_pickle('../data/pandas/rooms.pkl')
df_rooms

,room_id,room_number,floor,type,square_feet,basic_amenities,additional_amenities,max_occupancy,bed_type,view_type,accessibility,status,last_renovation,base_rate,max_rate
0,RM000001,101,1,Standard,410,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[City View],2,Queen,[City View],False,Available,2022-10-25,350,500
1,RM000002,102,1,Standard,361,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Evening Turndown Service],2,Queen,[Standard View],False,Available,2023-03-01,350,500
2,RM000003,103,1,Standard,431,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Courtyard View],2,Double Queen,[Courtyard View],False,Available,2022-08-10,350,500
3,RM000004,104,1,Standard,391,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[City View, Courtyard View, Evening Turndown S...",2,Queen,"[City View, Courtyard View]",True,Occupied,2023-09-06,350,500
4,RM000005,105,1,Standard,373,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[Courtyard View, City View, Evening Turndown S...",2,Double Queen,"[City View, Courtyard View]",True,Occupied,2024-04-13,350,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,RM000596,2026,20,Presidential Suite,2369,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Pool, Private Terrace, Luxury Car Ser...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2024-10-01,3500,5000
596,RM000597,2027,20,Presidential Suite,2020,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Terrace, Grand Piano, Private Gym Equ...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2024-04-17,3500,5000
597,RM000598,2028,20,Presidential Suite,1883,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Grand Piano, Private Terrace, Dedicated Conci...",6,King + Multiple Sofa Beds,[Standard View],False,Occupied,2022-03-09,3500,5000
598,RM000599,2029,20,Presidential Suite,1643,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Terrace, Private Chef Available, Priv...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2023-07-24,3500,5000


In [144]:
print(df_rooms.iloc[121])

room_id                                                          RM000122
room_number                                                           502
floor                                                                   5
type                                                               Deluxe
square_feet                                                           460
basic_amenities         [Air Conditioning, 55" Smart TV, Nespresso Mac...
additional_amenities                 [Soaking Tub, Pool View, Ocean View]
max_occupancy                                                           3
bed_type                                                     Double Queen
view_type                                         [Pool View, Ocean View]
accessibility                                                       False
status                                                           Occupied
last_renovation                                                2024-11-25
base_rate                             

Convert the room_id to an int for more efficient use in SQL.

In [158]:
df_rooms['room_id'] = df_rooms['room_id'].apply(lambda x: int(x[2:]))
df_rooms

,room_id,room_number,floor,type,square_feet,basic_amenities,additional_amenities,max_occupancy,bed_type,view_type,accessibility,status,last_renovation,base_rate,max_rate
0,1,101,1,Standard,410,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[City View],2,Queen,[City View],False,Available,2022-10-25,350,500
1,2,102,1,Standard,361,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Evening Turndown Service],2,Queen,[Standard View],False,Available,2023-03-01,350,500
2,3,103,1,Standard,431,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Courtyard View],2,Double Queen,[Courtyard View],False,Available,2022-08-10,350,500
3,4,104,1,Standard,391,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[City View, Courtyard View, Evening Turndown S...",2,Queen,"[City View, Courtyard View]",True,Occupied,2023-09-06,350,500
4,5,105,1,Standard,373,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[Courtyard View, City View, Evening Turndown S...",2,Double Queen,"[City View, Courtyard View]",True,Occupied,2024-04-13,350,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,2026,20,Presidential Suite,2369,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Pool, Private Terrace, Luxury Car Ser...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2024-10-01,3500,5000
596,597,2027,20,Presidential Suite,2020,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Terrace, Grand Piano, Private Gym Equ...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2024-04-17,3500,5000
597,598,2028,20,Presidential Suite,1883,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Grand Piano, Private Terrace, Dedicated Conci...",6,King + Multiple Sofa Beds,[Standard View],False,Occupied,2022-03-09,3500,5000
598,599,2029,20,Presidential Suite,1643,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Terrace, Private Chef Available, Priv...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2023-07-24,3500,5000


Get room types to create enum

In [92]:
room_type = df_rooms['type'].unique().tolist()
room_type_str = "('" + "', '".join(room_type) + "')"
room_type_str

"('Standard', 'Deluxe', 'Suite', 'Presidential Suite')"

Get room bed types to create enum

In [93]:
room_bed_type = df_rooms['bed_type'].unique().tolist()
room_bed_type_str = "('" + "', '".join(room_bed_type) + "')"
room_bed_type_str

"('Queen', 'Double Queen', 'King', 'Double King', 'King + Sofa Bed', 'King + Multiple Sofa Beds')"

Get statuses to create enum

In [199]:
room_status_type = df_rooms['status'].unique().tolist()
room_status_type_str = "('" + "', '".join(room_status_type) + "')"
room_status_type_str

"('Available', 'Occupied', 'Maintenance')"

Check that room_ids are sequential and start with 1

In [95]:
assert sum(df_rooms['room_id'].iloc[1:].to_numpy() - df_rooms['room_id'].iloc[:-1].to_numpy()) == df_rooms.shape[0]-1
assert df_rooms['room_id'].iloc[0] == 1

For some reason only the room_type enum needs special treatment (otherwise there's a cache error). I use more complicated ones for the bed_type and status columns, but they don't have a problem.

In [159]:
from enum import Enum

class RoomType(Enum):
    Standard = "Standard"
    Deluxe = "Deluxe"
    Suite = "Suite"
    Presidential_Suite = "Presidential Suite"
    
enum_room_type_mapping = {'Standard':RoomType.Standard, 'Deluxe':RoomType.Deluxe, 'Suite':RoomType.Suite, 'Presidential Suite':RoomType.Presidential_Suite}

sql_room_type_mapping = [
        (RoomType.Standard, 'Standard'),
        (RoomType.Deluxe, 'Deluxe'),
        (RoomType.Suite, 'Suite'),
        (RoomType.Presidential_Suite, 'Presidential Suite')
]

In [160]:
df_rooms['type'] = df_rooms['type'].apply(lambda x: enum_room_type_mapping[x])
df_rooms

,room_id,room_number,floor,type,square_feet,basic_amenities,additional_amenities,max_occupancy,bed_type,view_type,accessibility,status,last_renovation,base_rate,max_rate
0,1,101,1,RoomType.Standard,410,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[City View],2,Queen,[City View],False,Available,2022-10-25,350,500
1,2,102,1,RoomType.Standard,361,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Evening Turndown Service],2,Queen,[Standard View],False,Available,2023-03-01,350,500
2,3,103,1,RoomType.Standard,431,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Courtyard View],2,Double Queen,[Courtyard View],False,Available,2022-08-10,350,500
3,4,104,1,RoomType.Standard,391,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[City View, Courtyard View, Evening Turndown S...",2,Queen,"[City View, Courtyard View]",True,Occupied,2023-09-06,350,500
4,5,105,1,RoomType.Standard,373,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[Courtyard View, City View, Evening Turndown S...",2,Double Queen,"[City View, Courtyard View]",True,Occupied,2024-04-13,350,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,2026,20,RoomType.Presidential_Suite,2369,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Pool, Private Terrace, Luxury Car Ser...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2024-10-01,3500,5000
596,597,2027,20,RoomType.Presidential_Suite,2020,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Terrace, Grand Piano, Private Gym Equ...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2024-04-17,3500,5000
597,598,2028,20,RoomType.Presidential_Suite,1883,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Grand Piano, Private Terrace, Dedicated Conci...",6,King + Multiple Sofa Beds,[Standard View],False,Occupied,2022-03-09,3500,5000
598,599,2029,20,RoomType.Presidential_Suite,1643,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Terrace, Private Chef Available, Priv...",6,King + Multiple Sofa Beds,[Standard View],False,Maintenance,2023-07-24,3500,5000


Add table to Neon

In [201]:
from psycopg.types.enum import EnumInfo, register_enum

with psycopg.connect(neon_conn_string) as neon_conn:
    print("Connection established")
    # Open a cursor to perform database operations
    with neon_conn.cursor() as cur:
        # drop existing stuff (if exists)
        cur.execute("DROP TABLE IF EXISTS rooms CASCADE;")
        cur.execute("DROP TYPE IF EXISTS room_type CASCADE;")
        cur.execute("DROP TYPE IF EXISTS room_bed_type CASCADE;")
        cur.execute("DROP TYPE IF EXISTS room_status_type CASCADE;")
        # print("Finished dropping enums and table (if they existed).")
        
        # Define enums for type, bed_type, and status
        cur.execute(f"CREATE TYPE room_type AS ENUM {room_type_str};")
        cur.execute(f"CREATE TYPE room_bed_type AS ENUM {room_bed_type_str};")
        cur.execute(f"CREATE TYPE room_status_type AS ENUM {room_status_type_str};")

        # Create a new table
        cur.execute("""
            CREATE TABLE rooms (
                room_id INT GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
                room_number INT NOT NULL,
                floor INT NOT NULL,
                type room_type,
                square_feet INT,
                basic_amenities TEXT[],  -- Array data type for basic amenities
                additional_amenities TEXT[], -- Array data type for additional amenities
                max_occupancy INT,
                bed_type room_bed_type,
                view_type TEXT[],  -- Array data type for view type
                accessibility BOOLEAN,
                status room_status_type,
                last_renovation DATE,
                base_rate NUMERIC(10, 2),
                max_rate NUMERIC(10, 2)
            );
        """)
        print("Finished creating table.")
    
    # room_type enum special treatment
    info = EnumInfo.fetch(neon_conn, "room_type")
    register_enum(info, context=neon_conn, enum=RoomType, mapping=sql_room_type_mapping)
    
    with neon_conn.cursor() as cur:
        # Insert room availability data
        cur.executemany(
            f"INSERT INTO rooms VALUES (" + "%s, "*(len(df_rooms.columns)-1) + "%s);",
            df_rooms.values.tolist(),
        )
        print("Inserted data.")

Connection established
Finished creating table.
Inserted data.


## Room Availability table

Now add the room_availability table

In [165]:
df_room_availability = pd.read_pickle('../data/pandas/room_availability.pkl')
df_room_availability

,room_id,room_number,date,status,price,max_occupancy
0,RM000001,101,2025-01-04,Booked,NaN,2
1,RM000001,101,2025-01-05,Booked,NaN,2
2,RM000001,101,2025-01-06,Booked,NaN,2
3,RM000001,101,2025-01-07,Booked,NaN,2
4,RM000001,101,2025-01-08,Booked,NaN,2
...,...,...,...,...,...,...
218995,RM000600,2030,2025-12-30,Booked,NaN,6
218996,RM000600,2030,2025-12-31,Booked,NaN,6
218997,RM000600,2030,2026-01-01,Booked,NaN,6
218998,RM000600,2030,2026-01-02,Available,4617.6,6


Convert room_id to int for more efficient use in SQL and because used that way in rooms table.

In [166]:
df_room_availability['room_id'] = df_room_availability['room_id'].apply(lambda x: int(x[2:]))
df_room_availability

,room_id,room_number,date,status,price,max_occupancy
0,1,101,2025-01-04,Booked,NaN,2
1,1,101,2025-01-05,Booked,NaN,2
2,1,101,2025-01-06,Booked,NaN,2
3,1,101,2025-01-07,Booked,NaN,2
4,1,101,2025-01-08,Booked,NaN,2
...,...,...,...,...,...,...
218995,600,2030,2025-12-30,Booked,NaN,6
218996,600,2030,2025-12-31,Booked,NaN,6
218997,600,2030,2026-01-01,Booked,NaN,6
218998,600,2030,2026-01-02,Available,4617.6,6


In [12]:
df_room_availability.values.tolist()

[[1, 101, '2025-01-04', 'Booked', nan, 2],
 [1, 101, '2025-01-05', 'Booked', nan, 2],
 [1, 101, '2025-01-06', 'Booked', nan, 2],
 [1, 101, '2025-01-07', 'Booked', nan, 2],
 [1, 101, '2025-01-08', 'Booked', nan, 2],
 [1, 101, '2025-01-09', 'Booked', nan, 2],
 [1, 101, '2025-01-10', 'Booked', nan, 2],
 [1, 101, '2025-01-11', 'Booked', nan, 2],
 [1, 101, '2025-01-12', 'Booked', nan, 2],
 [1, 101, '2025-01-13', 'Booked', nan, 2],
 [1, 101, '2025-01-14', 'Booked', nan, 2],
 [1, 101, '2025-01-15', 'Booked', nan, 2],
 [1, 101, '2025-01-16', 'Booked', nan, 2],
 [1, 101, '2025-01-17', 'Booked', nan, 2],
 [1, 101, '2025-01-18', 'Booked', nan, 2],
 [1, 101, '2025-01-19', 'Booked', nan, 2],
 [1, 101, '2025-01-20', 'Available', 500.0, 2],
 [1, 101, '2025-01-21', 'Available', 500.0, 2],
 [1, 101, '2025-01-22', 'Available', 468.57, 2],
 [1, 101, '2025-01-23', 'Available', 485.14, 2],
 [1, 101, '2025-01-24', 'Available', 500.0, 2],
 [1, 101, '2025-01-25', 'Available', 500.0, 2],
 [1, 101, '2025-01-26'

Create the status enum string

In [171]:
availability_status_type = df_room_availability['status'].unique().tolist()
availability_status_type_str = "('" + "', '".join(availability_status_type) + "')"
availability_status_type_str

"('Booked', 'Available', 'Maintenance')"

This enum also has a problem with psycopg confusing it with the status enum for the rooms table.

In [183]:
from enum import Enum

class AvailabilityStatusType(Enum):
    Booked = "Booked"
    Available = "Available"
    Maintenance = "Maintenance"
    
enum_availability_status_type_mapping = {"Booked":AvailabilityStatusType.Booked, "Available":AvailabilityStatusType.Available, "Maintenance":AvailabilityStatusType.Maintenance}

In [184]:
df_room_availability['status'] = df_room_availability['status'].apply(lambda x: enum_availability_status_type_mapping[x])
df_room_availability

,room_id,room_number,date,status,price,max_occupancy
0,1,101,2025-01-04,AvailabilityStatusType.Booked,NaN,2
1,1,101,2025-01-05,AvailabilityStatusType.Booked,NaN,2
2,1,101,2025-01-06,AvailabilityStatusType.Booked,NaN,2
3,1,101,2025-01-07,AvailabilityStatusType.Booked,NaN,2
4,1,101,2025-01-08,AvailabilityStatusType.Booked,NaN,2
...,...,...,...,...,...,...
218995,600,2030,2025-12-30,AvailabilityStatusType.Booked,NaN,6
218996,600,2030,2025-12-31,AvailabilityStatusType.Booked,NaN,6
218997,600,2030,2026-01-01,AvailabilityStatusType.Booked,NaN,6
218998,600,2030,2026-01-02,AvailabilityStatusType.Available,4617.6,6


In [202]:

with psycopg.connect(neon_conn_string) as neon_conn:
    print("Connection established")
    # Open a cursor to perform database operations
    with neon_conn.cursor() as cur:
        # Drop the table if it already exists
        cur.execute("DROP TABLE IF EXISTS room_availability;")
        cur.execute("DROP TYPE IF EXISTS availability_status_type CASCADE;")
        print("Finished dropping table and enum (if they existed).")
        
        cur.execute(f"CREATE TYPE availability_status_type AS ENUM {availability_status_type_str};")
        # Create a new table
        cur.execute("""
            CREATE TABLE room_availability (
                id INT GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
                room_id INT NOT NULL,
                room_number INT NOT NULL,
                date DATE NOT NULL,
                status availability_status_type,
                price NUMERIC(8,2),
                max_occupancy INT,
                FOREIGN KEY (room_id) REFERENCES rooms(room_id)
            );
        """)
        print("Finished creating table.")
        
    # availability_status_type enum special treatment
    info = EnumInfo.fetch(neon_conn, "availability_status_type")
    register_enum(info, context=neon_conn, enum=AvailabilityStatusType)
    
    with neon_conn.cursor() as cur:
        # Insert room availability data
        cur.executemany(
            "INSERT INTO room_availability (room_id, room_number, date, status, price, max_occupancy) VALUES (%s, %s, %s, %s, %s, %s);",
            df_room_availability.values.tolist(),
        )
        print("Inserted data.")

Connection established
Finished dropping table and enum (if they existed).
Finished creating table.
Inserted data.


In [ ]:
try:
    with psycopg.connect(neon_conn_string) as neon_conn:
        print("Connection established")
        # Open a cursor to perform database operations
        with neon_conn.cursor() as cur:
            # Drop the table if it already exists
            cur.execute("SELECT COUNT(*) FROM room_availability")
            print(cur.fetchall())
except Exception as e:
    print("Connection failed.")
    print(e)

Connection established
[(219000,)]


In [10]:
with psycopg.connect(neon_conn_string) as neon_conn:
    print("Connection established")
    # Open a cursor to perform database operations
    with neon_conn.cursor() as cur:
        # Drop the table if it already exists
        cur.execute("""SELECT DISTINCT element FROM (
                SELECT unnest(basic_amenities) AS element
                FROM rooms
                UNION ALL
                SELECT unnest(additional_amenities) AS element
                FROM rooms);""")
        print(cur.fetchall())

Connection established
[('Executive Office',), ('High-Speed WiFi',), ('Welcome Amenity',), ('Private Chef Available',), ('Private Terrace',), ('Guest Bathroom',), ('Steam Room',), ('Living Room',), ('Luxury Welcome Amenity',), ('Bang & Olufsen Sound System',), ('Ultra-High-Speed WiFi',), ('Steam Shower',), ('Multiple 75" Smart TVs',), ('Sauna',), ('Panoramic Ocean View',), ('Soaking Tub',), ('Balcony',), ('Multiple Bathrooms',), ('Dining Area',), ('City View',), ('Dedicated Concierge',), ('Corner View',), ('Designer Slippers',), ('Ocean View',), ('Butler Service',), ('Formal Dining Room',), ('Private Bar',), ('Lounge Access',), ('Mini Fridge',), ('Frette Bathrobes',), ('Full Kitchen',), ('Nespresso Machine',), ('Premium Coffee Maker',), ('Wine Fridge',), ('Premium Bathrobes',), ('Kitchenette',), ('Private Gym Equipment',), ('Bathrobes',), ("Butler's Pantry",), ('Full-Size Refrigerator',), ('55" Smart TV',), ('Bluetooth Speaker',), ('Professional Coffee Bar',), ('Pool View',), ('Air Con

In [11]:
from openai import OpenAI

class NL2SQLAgent:
    def __init__(self, db_conn, llm_conn):
        self.db = db_conn
        self.llm = llm_conn
        self._get_enums()
        self._get_amenities()
        self._get_view_types()

    def _get_enums(self):
        with psycopg.connect(neon_conn_string) as neon_conn:
            with neon_conn.cursor() as cur:
                cur.execute("SELECT enum_range(NULL::room_type);")
                self.room_type_enum = cur.fetchall()[0][0]
                cur.execute("SELECT enum_range(NULL::room_bed_type);")
                self.room_bed_type_enum = cur.fetchall()[0][0]
                cur.execute("SELECT enum_range(NULL::room_status_type);")
                self.room_status_type_enum = cur.fetchall()[0][0]
                cur.execute("SELECT enum_range(NULL::availability_status_type);")
                self.availability_status_type_enum = cur.fetchall()[0][0]
                
    def _get_amenities(self):
        with psycopg.connect(neon_conn_string) as neon_conn:
            with neon_conn.cursor() as cur:
                cur.execute("SELECT DISTINCT unnest(basic_amenities) FROM rooms;")
                self.basic_amenities = cur.fetchall()
                cur.execute("SELECT DISTINCT unnest(additional_amenities) FROM rooms;")
                self.additional_amenities = cur.fetchall()
                
    def _get_view_types(self):
        with psycopg.connect(neon_conn_string) as neon_conn:
            with neon_conn.cursor() as cur:
                cur.execute("SELECT DISTINCT unnest(view_type) FROM rooms;")
                self.view_types = cur.fetchall()
                
    def handle_nl_query(self, query: str):
        instructions = f"""
            You are a SQL query writer. Given the user's input, write a query that, when
            passed to a PostgreSQL database, will gather the data necessary to answer
            the user's query.
            
            You have access to the rooms table, which provides information about the hotel
            with the following columns:
             - room_id: Primary key for the table
             - room_number: The room number
             - floor: The floor the room is on
             - type: The type of room, one of the SQL enum {self.room_type_enum}
             - square_feet: The size of the room in square feet
             - basic_amenities: Array of the basic amenities found in the room, chosen from the SQL list {self.basic_amenities}
             - additional_amenities: Array of additional amenities found in the room, chosen from the SQL list {self.additional_amenities}
             - max_occupancy: The maximum number of people allowed to stay in the room
             - bed_type: The type of bed(s) found in the room, one of the SQL enum {self.room_bed_type_enum}
             - view_type: Array of the views provided by the room, chosen from the SQL list {self.view_types}
             - accessibility: A boolean specifying whether the room is handicapped accessible
             - status: Ignore this
             - last_renovation: When the room was last renovated
             - base_rate: The base price a night in the room can be booked for
             - max_rate: The maximum price charge to book the room for the night
            
            You have access to the table room_availability, which provides information
            on the availability of rooms in a hotel, with the following columns:
             - room_id: Foreign key referring to the room_id in the rooms table
             - room_number: The room number
             - date: The date the availability information in this row is for
             - status: Whether the room is available or not. Options are Available, Booked, and Maintenance.
             - price: The current price to book the room for that date
             - max_occupancy: The maximum number of people who can stay in the room
        """
        
        return self.llm.responses.create(
            model="gpt-5-mini",
            instructions=instructions,
            input=query,
        )
    
neon_conn = psycopg.connect(neon_conn_string)
openai_conn = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
nl_2_sql_agent = NL2SQLAgent(neon_conn, openai_conn)

In [12]:
def parse_to_sql(nl_query: str) -> str:
    return nl_2_sql_agent.handle_nl_query(nl_query)

In [17]:
def test_nl2sql():
    query = 'Show me rooms with a view of the courtyard and a 55" TV'
    sql = parse_to_sql(query)
    return sql
    
response_query = test_nl2sql()

In [18]:
print(response_query.output_text)

SELECT
  room_id,
  room_number,
  floor,
  type,
  square_feet,
  bed_type,
  max_occupancy,
  accessibility,
  last_renovation,
  base_rate,
  max_rate,
  basic_amenities,
  additional_amenities,
  view_type
FROM rooms
WHERE 'Courtyard View' = ANY(view_type)
  AND '55" Smart TV' = ANY(basic_amenities)
ORDER BY floor, room_number;


In [15]:
neon_conn.execute(response_query.output_text).fetchall()

[(109,
  419,
  4,
  'Standard',
  'Double Queen',
  2,
  2,
  430,
  Decimal('350.00'),
  Decimal('500.00'),
  Decimal('386.26'),
  ['City View', 'Courtyard View'],
  ['Air Conditioning',
   'Smart TV',
   'Premium Coffee Maker',
   'Mini Fridge',
   'Hair Dryer',
   'In-Room Safe',
   'Work Desk',
   'High-Speed WiFi',
   'Bathrobes',
   'Slippers'],
  ['City View', 'Evening Turndown Service', 'Courtyard View'],
  False,
  datetime.date(2022, 9, 9)),
 (45,
  215,
  2,
  'Standard',
  'Queen',
  2,
  2,
  442,
  Decimal('350.00'),
  Decimal('500.00'),
  Decimal('388.67'),
  ['Courtyard View'],
  ['Air Conditioning',
   'Smart TV',
   'Premium Coffee Maker',
   'Mini Fridge',
   'Hair Dryer',
   'In-Room Safe',
   'Work Desk',
   'High-Speed WiFi',
   'Bathrobes',
   'Slippers'],
  ['Courtyard View'],
  False,
  datetime.date(2024, 4, 8)),
 (43,
  213,
  2,
  'Standard',
  'Queen',
  2,
  2,
  371,
  Decimal('350.00'),
  Decimal('500.00'),
  Decimal('390.95'),
  ['City View', 'Courtyar

,room_number,floor,type,square_feet,basic_amenities,additional_amenities,max_occupancy,bed_type,view_type,accessibility,status,last_renovation,base_rate,max_rate
room_id,,,,,,,,,,,,,,
RM000001,101,1,Standard,410,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[City View],2,Queen,City View,False,Available,2022-10-25,350,500
RM000002,102,1,Standard,361,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Evening Turndown Service],2,Queen,Standard View,False,Available,2023-03-01,350,500
RM000003,103,1,Standard,431,"[Air Conditioning, Smart TV, Premium Coffee Ma...",[Courtyard View],2,Double Queen,Courtyard View,False,Available,2022-08-10,350,500
RM000004,104,1,Standard,391,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[City View, Courtyard View, Evening Turndown S...",2,Queen,Courtyard View,True,Occupied,2023-09-06,350,500
RM000005,105,1,Standard,373,"[Air Conditioning, Smart TV, Premium Coffee Ma...","[Courtyard View, City View, Evening Turndown S...",2,Double Queen,Evening Turndown Service,True,Occupied,2024-04-13,350,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RM000596,2026,20,Presidential Suite,2369,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Pool, Private Terrace, Luxury Car Ser...",6,King + Multiple Sofa Beds,Standard View,False,Maintenance,2024-10-01,3500,5000
RM000597,2027,20,Presidential Suite,2020,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Private Terrace, Grand Piano, Private Gym Equ...",6,King + Multiple Sofa Beds,Standard View,False,Maintenance,2024-04-17,3500,5000
RM000598,2028,20,Presidential Suite,1883,"[Air Conditioning, Multiple 75"" Smart TVs, Pro...","[Grand Piano, Private Terrace, Dedicated Conci...",6,King + Multiple Sofa Beds,Standard View,False,Occupied,2022-03-09,3500,5000
